In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob,re,time,os
from multiprocessing import Pool
from tqdm import tqdm

In [21]:
binsize = 1000000
def calc_mad(sample):
#     try:
        sampledir='./%s' % sample
        mad_file = sampledir+'/SegMAD.gz'
        segnorm = pd.read_csv(sampledir+'/SegNorm.gz', sep='\t',index_col=['CHR','START','END'])
        segnorm=segnorm.astype(float)
        segnorm = segnorm.drop(index=['chrX'],level='CHR')
        segnorm = segnorm.loc[segnorm.index.get_level_values('END')-segnorm.index.get_level_values('START')==binsize-1]
        segnorm = segnorm.sort_index()
        print(f'Keeping {segnorm.shape[0]} bins')

        # Keep only the bins with the normative binsize
        mad = segnorm.diff(axis=0).abs().median(axis=0)
        mad.to_csv(mad_file,sep='\t',header=False)
    #     print('Finished %s' % sample)
        return mad
#     except:
#         print(f'Error on {sample}')
#         return segnorm

In [22]:
# Load Ginkgo results for a set of cells
datadir='./*/status.xml'
ginkgo_list = glob.glob(datadir)
ginkgo_list = [i.split('/')[-2] for i in ginkgo_list]
ginkgo_list = np.sort(ginkgo_list)
print(len(ginkgo_list))

6


In [23]:
ginkgo_list[0]

'Luo2017_human'

In [24]:
calc_mad(ginkgo_list[0])

Keeping 2833 bins


Pool_1000_AD008    0.127231
Pool_1000_AD010    0.136222
Pool_1001_AD008    0.121418
Pool_1001_AD010    0.126582
Pool_1002_AD008    0.129288
                     ...   
Pool_999_AD008     0.140190
Pool_999_AD010     0.128512
Pool_99_AD008      0.139017
Pool_99_AD010      0.129551
Pool_9_AD008       0.123475
Length: 2888, dtype: float64

In [4]:
with Pool() as p:
    x = list(tqdm(p.imap(calc_mad, ginkgo_list),total=len(ginkgo_list)))

  0%|          | 0/6 [00:00<?, ?it/s]

Error on MB_EB
Error on mCTN-1
Error on MB_EA
Error on MB_v1
Error on Luo2017_human
Error on mCTN-2


100%|██████████| 6/6 [00:03<00:00,  1.60it/s]
